THis is web scrapping project to get data from yallakora website and send save it in google sheet and send the today's matches to my friends  

Planning:

getting todays date

link

make exception to make sure the webpage is already working well

get the data

parse data

identify which data i want

save the data i want into something

save data into google sheet

find automatic method to run the program and sending today's matches daily to someone

In [1]:
from datetime import date
import requests
from bs4 import BeautifulSoup
from requests.exceptions import RequestException
import lxml
import pandas as pd

In [2]:
today = date.today()
# mm/dd/y
date = today.strftime("%m/%d/%Y")
print("date =", date)


# dd/mm/YY
#d1 = today.strftime("%d/%m/%Y")
#print("d1 =", d1)

# Textual month, day and year
#d2 = today.strftime("%B %d, %Y")
#print("d2 =", d2)

# Month abbreviation, day and year
#d4 = today.strftime("%b-%d-%y")
#print("d4 =", d4)


date = 08/26/2024


In [3]:
#date = 12/17/2022

In [4]:
page = requests.get(f"https://www.yallakora.com/match-center/%D9%85%D8%B1%D9%83%D8%B2-%D8%A7%D9%84%D9%85%D8%A8%D8%A7%D8%B1%D9%8A%D8%A7%D8%AA?date={date}#")

In [6]:
"""
# URL of the webpage to check
url = page

try:
    # Send a GET request to the URL
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Example: Check if title tag exists
    title_tag = soup.find('title')
    if title_tag:
        print(f"Webpage '{url}' is working properly. Title: {title_tag.get_text()}")
    else:
        print(f"Webpage '{url}' is missing a title tag.")

except RequestException as e:
    print(f"Error accessing webpage '{url}': {e}")


"""


'\n# URL of the webpage to check\nurl = page\n\ntry:\n    # Send a GET request to the URL\n    response = requests.get(url)\n    response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)\n\n    # Parse the HTML content using BeautifulSoup\n    soup = BeautifulSoup(response.content, \'html.parser\')\n\n    # Example: Check if title tag exists\n    title_tag = soup.find(\'title\')\n    if title_tag:\n        print(f"Webpage \'{url}\' is working properly. Title: {title_tag.get_text()}")\n    else:\n        print(f"Webpage \'{url}\' is missing a title tag.")\n\nexcept RequestException as e:\n    print(f"Error accessing webpage \'{url}\': {e}")\n\n\n'

In [7]:
def main(page):
    src = page.content
    soup = BeautifulSoup(src, "html.parser")
    matches_details = []

    championships = soup.find_all("div", {'class': 'matchCard'}) # Find all elements with class 'matchCard'
    if not championships:
        print("No championships found")
        return

    def get_match_info(championships):
        championship_title = championships.contents[1].find("h2").text.strip()
        all_matches = championships.find_all("div", {'class': 'liItem'})
        number_of_matches = len(all_matches)
        #print(number_of_matches)

        for i in range(number_of_matches):
            #teams name
            team_A = all_matches[i].find('div', {'class': 'teamA'}).text.strip()
            team_B = all_matches[i].find('div', {'class': 'teamB'}).text.strip()

            #get score
            match_result = all_matches[i].find('div', {'class': 'MResult'}).find_all('span', {'class': 'score'})
            score = f"{match_result[0].text.strip()} - {match_result[1].text.strip()}"

            #get match time
            match_time = all_matches[i].find('div', {'class': 'MResult'}).find('span', {'class': 'time'}).text.strip()

            # add info to match details
            matches_details.append({
                "Championship": championship_title,
                "Team A": team_A,
                "Team B": team_B,
                "Match Time": match_time,
                "Score": score
            })


    for championship in championships:
        get_match_info(championship)

    return (matches_details)

match_details = main(page)

# Create DataFrame
df = pd.DataFrame(match_details)

# Print DataFrame
df


,Championship,Team A,Team B,Match Time,Score
0,الدوري الإسباني,فياريال,سيلتا فيجو,22:30,- - -
1,الدوري الإيطالي,كالياري,كومو,19:30,0 - 0
2,الدوري الإيطالي,هيلاس فيرونا,يوفنتوس,21:45,- - -


In [8]:
pip install retrying

In [9]:
 # Import necessary libraries
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders


# Process your DataFrame as needed
# Example: Get today's data
today_data = df[df['Date'] == pd.Timestamp.today().strftime('%Y-%m-%d')]

# Example of sending an email with an attachment
def send_email(receiver_email, subject, body, attachment_path=None):
    sender_email = "whirlpoolgalaxy5@gmail.com"
    sender_password = "vqcvkllttnpgwrfj"

    # Setup the MIME
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email
    message['Subject'] = subject

    # Add body to email
    message.attach(MIMEText(body, 'plain'))

    # Add attachment
    if attachment_path:
        attachment = open(attachment_path, "rb")
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename= {attachment_path.split("/")[-1]}', )
        message.attach(part)

    # Create SMTP session for sending the mail
    session = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
    session.starttls()  # enable security
    session.login(sender_email, sender_password)  # login with mail_id and password
    text = message.as_string()
    session.sendmail(sender_email, receiver_email, text)
    first

receiver_email = "salehbasry3@gmail.com"
send_email(receiver_email, subject, body, attachment_path=None)

KeyError: 'Date'